# Linear Programmin' using Python Tools

- Mirko Stojiljković (2020). Hands-On Linear Programming: Optimization With Python [https://realpython.com/linear-programming-python/](https://realpython.com/linear-programming-python/)

In [311]:
import math
verbose = False #Set to True for lots-o-output
# build the cost model
# build the cost coefficient vector
xdist = [1,3,5,7,9] # position of grid centers in horozontal direction
ydist = [1,3,5,7,9] # position of grid centers in vertical direction
x0 = 5 # cell center Cell 18 - horizontal
y0 = 3 # cell center Cell 18 - vertical
cost = [] # empty list to store the cost coefficients
# cell location map
cellloc = [[1,5],[2,5],[3,5],[4,5],[5,5],\
           [1,4],[2,4],[3,4],[4,4],[5,4],\
           [1,3],[2,3],[3,3],[4,3],[5,3],\
           [1,2],[2,2],[3,2],[4,2],[5,2],\
           [1,1],[2,1],[3,1],[4,1],[5,1] ]
for icell in range(25):
    part1 = xdist[cellloc[icell][0] - 1]-x0
    part2 = ydist[cellloc[icell][1] - 1]-y0
    distance = math.sqrt((part1)**2 + (part2)**2)
    cost.append( 1.0 + 0.5 * distance)

# cost now contains cost coefficients for the LP, decisions are the pump rates, this will be passed to the objective function

In [312]:
# Read the influence table
infile='influence-table.out'
localfile=open(infile,"r")
tableau = [] # 25 rows in the influence matrix, goes to top of the LP structure
for ir in range(16):
    tableau.append([-1*float(n) for n in localfile.readline().strip().split()])
for ir in range(16,25):
    tableau.append([float(n) for n in localfile.readline().strip().split()])
demand = [1.0 for jc in range(25)] # a row of ones!
tableau.append(demand)
# force zeros pumping in cells 21-25
row27 = [0.0 for jc in range(25)]
row28 = [0.0 for jc in range(25)]
row29 = [0.0 for jc in range(25)]
row30 = [0.0 for jc in range(25)]
row31 = [0.0 for jc in range(25)]
row27[20]=1.0
row28[21]=1.0
row29[22]=1.0
row30[23]=1.0
row31[24]=1.0
# append these to the tableau
tableau.append(row27)
tableau.append(row28)
tableau.append(row29)
tableau.append(row30)
tableau.append(row31)
# print the tableau
if verbose:
    for i in range(len(tableau)):
        print(tableau[i][:])
# the tableau contains the LP constraint matrix, now build the corresponding RHS

In [313]:
# Build the constraint RHS vector (SciPy uses less-than for inequality)
# drawdown constraints
# demand constraints
# non-negative constraints

rhs_ineq = [0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6.22,6.22,6.22,6.22,6.22,2.15,2.15,2.15,2.15,2.15] # drawdown constraints
#rhs_ineq.append(-7.00) # minimum demand constraint (because uses <=) change sense and sign
rhs_eq = [7.00,0.00,0.00,0.00,0.00,0.00] # force zero cells 21-25

# now list slice to populate rest

lhs_ineq = tableau[0:25][:] # part of tableau for drawdown and demand
lhs_eq = tableau[25:31][:] # part of tableau for forced zeros
verbose=False
# print the tableau
if verbose:
    print('Inequality RHS')
    for i in range(len(lhs_ineq)):
        print(lhs_ineq[i][:])
    print('Equality RHS')
    for i in range(len(lhs_eq)):
        print(lhs_eq[i][:])
# the pieces are in SciPy structure, now complete the LP
from scipy.optimize import linprog # load the package

In [314]:
myoptions=dict({"maxiter":1000,"tol": 1e-9,"autoscale":False})
#myoptions

In [315]:
bnd = [(0, float("inf")) for i in range(25)] # set bounds 0-infnty

In [316]:
opt = linprog(c=cost, A_ub=lhs_ineq, b_ub=rhs_ineq,
...               A_eq=lhs_eq, b_eq=rhs_eq, bounds=bnd,
...               method="interior-point",options=myoptions)

In [317]:
opt

     con: array([-6.04849504e-12,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00])
     fun: 13.728730294265585
 message: 'Optimization terminated successfully.'
     nit: 9
   slack: array([ 5.81993176e+00,  5.83480665e+00,  5.82281159e+00,  5.77337363e+00,
        5.71189437e+00,  5.80505687e+00,  5.86167661e+00,  5.86025449e+00,
        5.78541492e+00,  5.65041511e+00,  5.73356224e+00,  5.94658842e+00,
        5.97111484e+00,  5.85761645e+00,  5.45393604e+00,  1.16690414e+01,
       -1.09103837e-11, -1.05400133e-11, -8.62154792e-12,  1.36622343e+00,
        2.55134890e-01,  9.53755983e-02,  7.39280779e-02,  1.27307668e-01,
        4.13517955e-01])
  status: 0
 success: True
       x: array([5.03388415e-13, 5.82334036e-13, 5.90060621e-13, 5.28373055e-13,
       4.43867951e-13, 7.57559341e-13, 9.95255910e-13, 6.77334125e-14,
       5.18066124e-14, 5.72105940e-13, 1.40109854e-12, 3.15116708e-12,
       7.91458700e-12, 2.06070916e-12, 4.391986

In [318]:
# check for solution
if opt.status == 0:
    print('Optimal Solution Found')
    print('Cost : ',round(opt.fun,2))
    sum = 0.0
    for i in range(25):
        print('Pumpage Cell',i+1,' = ',round(opt.x[i],3))
        sum=sum+opt.x[i]
    print('Total Pumpage :',round(sum,3))

Optimal Solution Found
Cost :  13.73
Pumpage Cell 1  =  0.0
Pumpage Cell 2  =  0.0
Pumpage Cell 3  =  0.0
Pumpage Cell 4  =  0.0
Pumpage Cell 5  =  0.0
Pumpage Cell 6  =  0.0
Pumpage Cell 7  =  0.0
Pumpage Cell 8  =  0.0
Pumpage Cell 9  =  0.0
Pumpage Cell 10  =  0.0
Pumpage Cell 11  =  0.0
Pumpage Cell 12  =  0.0
Pumpage Cell 13  =  0.0
Pumpage Cell 14  =  0.0
Pumpage Cell 15  =  0.0
Pumpage Cell 16  =  0.912
Pumpage Cell 17  =  1.902
Pumpage Cell 18  =  1.603
Pumpage Cell 19  =  2.163
Pumpage Cell 20  =  0.42
Pumpage Cell 21  =  0.0
Pumpage Cell 22  =  0.0
Pumpage Cell 23  =  0.0
Pumpage Cell 24  =  0.0
Pumpage Cell 25  =  0.0
Total Pumpage : 7.0


In [320]:
# convert solution into input file fragment for a simulation run
recharge = 0.1 #m/yr
pumpage = 1.0e6 #Mm^3/yr
dxdy = 2000*2000
Rin = dxdy*recharge

for i in range(25):
        print(round(opt.x[i]*1e6-Rin,3))
Rin

-400000.0
-400000.0
-400000.0
-400000.0
-400000.0
-400000.0
-400000.0
-400000.0
-400000.0
-400000.0
-400000.0
-400000.0
-400000.0
-400000.0
-400000.0
512024.387
1501557.195
1203376.832
1762958.847
20082.739
-400000.0
-400000.0
-400000.0
-400000.0
-400000.0


400000.0

In [284]:
(1.0)/dxdy-Rin

-399999.99999975

In [90]:
bnd

[(0, inf),
 (0, inf),
 (0, inf),
 (0, inf),
 (0, inf),
 (0, inf),
 (0, inf),
 (0, inf),
 (0, inf),
 (0, inf),
 (0, inf),
 (0, inf),
 (0, inf),
 (0, inf),
 (0, inf),
 (0, inf),
 (0, inf),
 (0, inf),
 (0, inf),
 (0, inf),
 (0, inf),
 (0, inf),
 (0, inf),
 (0, inf),
 (0, inf)]

In [73]:
from scipy.optimize import linprog

In [84]:
obj = [-1, -2]
lhs_ineq = [[ 2,  1], [-4,  5],  [ 1, -2]]  # Yellow constraint left side
rhs_ineq = [20, 10, 2]  # Yellow constraint right side
lhs_eq = [[-1, 5]]  # Green constraint left side
rhs_eq = [15]       # Green constraint right side


In [79]:
bnd = [(0, float("inf")),(0, float("inf"))]  

In [236]:
type(opt)

scipy.optimize.optimize.OptimizeResult

In [51]:
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

In [52]:
# Create the model
model = LpProblem(name="small-problem", sense=LpMaximize)
# Initialize the decision variables
x = LpVariable(name="x", lowBound=0)
y = LpVariable(name="y", lowBound=0)
# Add the objective function to the model
obj_func = x + 2 * y
model += obj_func
# Add the constraints to the model
model += (2 * x + y <= 20, "red_constraint")
model += (4 * x - 5 * y >= -10, "blue_constraint")
model += (-x + 2 * y >= -2, "yellow_constraint")
model += (-x + 5 * y == 15, "green_constraint")

In [53]:
# check model define
model

small-problem:
MAXIMIZE
1*x + 2*y + 0
SUBJECT TO
red_constraint: 2 x + y <= 20

blue_constraint: 4 x - 5 y >= -10

yellow_constraint: - x + 2 y >= -2

green_constraint: - x + 5 y = 15

VARIABLES
x Continuous
y Continuous

In [54]:
model.variables()

[x, y]

In [56]:
model.solve()

OSError: [Errno 8] Exec format error: '/opt/jupyterhub/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc'